<a href="https://colab.research.google.com/github/liuxx479/games/blob/master/unfair_coins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from pylab import *
%matplotlib inline

In [ ]:
####### set up the coins

coin1A=lambda: rand()<0.51
coin1B=lambda: rand()<0.49

coin2A=lambda: rand()<0.01
coin2B=lambda: rand()<0.02